---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

In [2]:
df = pd.read_csv('train.csv', encoding = "ISO-8859-1")

df.index = df['ticket_id']
df.head()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
ticket_id,,,,,,,,,,,,,,,,,,,,,
22056,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
27586,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
22062,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
22084,22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
22093,22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN


In [5]:
df[df.columns[df.dtypes=='float64']].columns.tolist()

['violation_street_number',
 'violation_zip_code',
 'mailing_address_str_number',
 'fine_amount',
 'admin_fee',
 'state_fee',
 'late_fee',
 'discount_amount',
 'clean_up_cost',
 'judgment_amount',
 'payment_amount',
 'balance_due',
 'compliance']

In [6]:
df[df.columns[df.dtypes!='float64']].columns.tolist()

['ticket_id',
 'agency_name',
 'inspector_name',
 'violator_name',
 'violation_street_name',
 'mailing_address_str_name',
 'city',
 'state',
 'zip_code',
 'non_us_str_code',
 'country',
 'ticket_issued_date',
 'hearing_date',
 'violation_code',
 'violation_description',
 'disposition',
 'payment_date',
 'payment_status',
 'collection_status',
 'grafitti_status',
 'compliance_detail']

In [8]:
df.compliance = df.compliance.fillna(value=-1)
df = df[df.compliance != -1]

df_val = pd.read_csv('readonly/test.csv', encoding = "ISO-8859-1")
df_val.index = df_val['ticket_id']
df_val.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status
ticket_id,,,,,,,,,,,,,,,,,,,,,
284932,284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,...,Failure to secure City or Private solid waste ...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
285362,285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN
285361,285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Improper placement of Courville container betw...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN
285338,285338,Department of Public Works,"Talbert, Reginald","HARABEDIEN, POPKIN",1835.0,CENTRAL,NaN,2246,NELSON,WOODHAVEN,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
285346,285346,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,...,Violation of time limit for approved container...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN


In [9]:
def clean_var(dataset):
    dataset.drop([
    'agency_name',
     'inspector_name',
     'violator_name',
     'violation_street_name',
     'mailing_address_str_name',
     'city',
     'state',
     'zip_code',
     'non_us_str_code',
     'country',
     'ticket_issued_date',
     'hearing_date',
     'violation_description',
     'grafitti_status',

     'violation_street_number',
     'violation_zip_code',
     'mailing_address_str_number',
     'disposition',
     'violation_code'

            ], axis = 1, inplace=True)
    return dataset

clean_var(df).drop(['payment_date', 'payment_status', 'collection_status', 'compliance_detail',
                   'payment_amount','balance_due'
                   ], axis = 1, inplace=True)
clean_var(df_val)

,ticket_id,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount
ticket_id,,,,,,,,
284932,284932,200.0,20.0,10.0,20.0,0.0,0.0,250.0
285362,285362,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0
285361,285361,100.0,20.0,10.0,10.0,0.0,0.0,140.0
285338,285338,200.0,20.0,10.0,20.0,0.0,0.0,250.0
285346,285346,100.0,20.0,10.0,10.0,0.0,0.0,140.0
285345,285345,200.0,20.0,10.0,20.0,0.0,0.0,250.0
285347,285347,50.0,20.0,10.0,5.0,0.0,0.0,85.0
285342,285342,200.0,20.0,10.0,0.0,0.0,0.0,230.0
285530,285530,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0


In [10]:
X = df.drop('compliance',axis=1)
X.fillna(value = -1)

y = df.compliance

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

In [11]:
clf = GradientBoostingClassifier(learning_rate = 0.03, max_depth = 4, random_state = 0).fit(X_train, y_train)
X_predict = clf.predict_proba(df_val)
    
clf = GradientBoostingClassifier(learning_rate = 0.03, max_depth = 4, random_state = 0).fit(X_train, y_train)
print('Accuracy of GBDT classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of GBDT classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

Accuracy of GBDT classifier on training set: 0.94
Accuracy of GBDT classifier on test set: 0.93


In [57]:
y_score = clf.predict(X_test)

fpr, tpr, _ = roc_curve(y_test, y_score)

roc_auc = auc(fpr, tpr)

print(roc_auc)

0.795316890929


In [167]:
import pandas as pd
import numpy as np

def blight_model():
    
    # Your code here
    import math
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.model_selection import train_test_split
    
    df = pd.read_csv('train.csv', encoding = "ISO-8859-1")
    df.index = df['ticket_id']

    df.compliance = df.compliance.fillna(value=-1)
    df = df[df.compliance != -1]

    df_val = pd.read_csv('test.csv', encoding = "ISO-8859-1")
    df_val.index = df_val['ticket_id']
    
    def clean_var(dataset):
        dataset.drop([
        'agency_name',
         'inspector_name',
         'violator_name',
         'violation_street_name',
         'mailing_address_str_name',
         'city',
         'state',
         'zip_code',
         'non_us_str_code',
         'country',
         'ticket_issued_date',
         'hearing_date',
         'violation_description',
         'grafitti_status',

         'violation_street_number',
         'violation_zip_code',
         'mailing_address_str_number',
         'disposition',
         'violation_code'
                      
                ], axis = 1, inplace=True)
        return dataset
        
    clean_var(df).drop(['payment_date', 'payment_status', 'collection_status', 'compliance_detail',
                       'payment_amount','balance_due'
                       ], axis = 1, inplace=True)
    clean_var(df_val)

    X = df.drop('compliance',axis=1)
    X.fillna(value = -1)

    y = df.compliance

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

#     def encode(input_var):
#         from sklearn.preprocessing import LabelEncoder
#         label_encoder = LabelEncoder()
#         label_encoder.fit(X_train[input_var].append(X_test[input_var], ignore_index=True))
#         X_train[input_var] = label_encoder.transform(X_train[input_var])
#         X_test[input_var] = label_encoder.transform(X_test[input_var])
#         df_val[input_var] = label_encoder.transform(df_val[input_var])
#     encode('disposition')
#     encode('violation_code')
    
    clf = GradientBoostingClassifier(learning_rate = 0.03, max_depth = 4, random_state = 0).fit(X_train, y_train)
    X_predict = clf.predict_proba(df_val)
    ans = pd.Series(data = X_predict[:,1], index = df_val['ticket_id'], dtype='float32')
    
    return  ans # Your answer here

In [168]:
blight_model()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


(61001,)


ticket_id
284932    0.116894
285362    0.028375
285361    0.119186
285338    0.116894
285346    0.119186
285345    0.116894
285347    0.114053
285342    0.428447
285530    0.028375
284989    0.031146
285344    0.114053
285343    0.028375
285340    0.028375
285341    0.114053
285349    0.119186
285348    0.116894
284991    0.031146
285532    0.031146
285406    0.031146
285001    0.031146
285006    0.028375
285405    0.028375
285337    0.031146
285496    0.114053
285497    0.116894
285378    0.028375
285589    0.031146
285585    0.116894
285501    0.119186
285581    0.028375
            ...   
376367    0.039593
376366    0.096079
376362    0.096079
376363    0.281643
376365    0.039593
376364    0.096079
376228    0.096079
376265    0.096079
376286    0.428447
376320    0.096079
376314    0.096079
376327    0.428447
376385    0.428447
376435    0.459142
376370    0.428447
376434    0.276049
376459    0.286117
376478    0.028734
376473    0.096079
376484    0.039593
376482    0.039593
37